<a href="https://colab.research.google.com/github/alecoder1/6-Nations-Rugby-Predictor/blob/main/6_Nations_rugby_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 6 NATIONS PREDICTOR

- I plan to try and predict the game results using the following data.
      - The teams ranking points as ranked by world rugny at the beginning of the tornament.
      - Home advantage (using the non - US sysetem of putting the home team first).
      - Historic results in the tournament.
      - Previous matchups in the tournament.
- The matches dataset contain all matches played from 2004 but that coontains duplication as it contains all the games Engald played in 2004 and then all the games France played and so on.

- Standing contains the year and the ranking pints for the team at the start of the tournament. This was only started in 2003 after 6 nations so 2004 is the first tournament we look at.

- Home_matchs_with_weather contains the matches with the air pressure, wind gust speed and 'deels like' temperature for the game.
  

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as DT
pd.set_option("display.max_rows", None)

## Loading the dataset

In [5]:
home_matches = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/Data/HomeData.csv")
rating = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/Data/RugbyStats (1).csv")
target = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/Data/target (2).csv")
home_matches_with_weather = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/Data/home_matches_with_weather.csv")

In [6]:
home_matches.head()

,Team,Diff,Opposition,Ground,Match Date
0,Wales,8,v Italy,Millennium Stadium,1 Feb 2014
1,France,2,v England,Stade de France,1 Feb 2014
2,France,-5,v Wales,Stade de France,1 Feb 2019
3,Ireland,7,v Scotland,Lansdowne Road,1 Feb 2020
4,Wales,42,v Italy,Millennium Stadium,1 Feb 2020


- Team - Home Team
- Diff - Target variable and is the match result. Positive means home win and negative is lozz and in favour of the away team.
- Opposition - away team
- Ground - stadium games were played at.
Match Date - match date. In 2020 some games were playd in later months due to covid-19.

In [7]:
rating.head(10)

,Member Union,Tier,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,ENGLAND,1,High Performance,93.99,86.59,83.27,78.61,85.57,80.87,80.72,82.48,81.58,83.90,85.70,85.68,79.76,90.47,90.88,86.23,88.73
1,IRELAND,1,High Performance,80.53,82.61,80.03,84.67,78.67,78.51,84.60,81.79,80.65,80.22,79.33,82.13,81.17,84.63,86.40,91.18,84.24
2,WALES,1,High Performance,76.92,76.91,82.54,76.87,74.17,81.03,79.89,77.04,79.61,78.39,80.19,80.70,83.49,82.56,82.09,87.26,79.77
3,FRANCE,1,High Performance,82.71,83.70,86.10,85.87,80.36,79.51,82.41,81.66,84.70,85.07,80.87,80.07,79.77,80.14,78.11,77.34,85.66
4,SCOTLAND,1,High Performance,76.36,74.65,73.52,76.73,76.70,76.75,76.59,81.20,76.20,75.83,76.56,75.16,78.01,80.71,84.13,81.86,81.21
5,ITALY,1,High Performance,71.88,72.51,72.18,72.79,73.29,74.56,72.59,73.31,73.99,76.24,73.13,72.05,72.79,72.50,71.27,72.76,70.88


- Member Union - Team
- Tier - world rugby is divided into tiers. Italy were in a lower tier and moved into the top tier in 2000. The gulf between the two tiers is large and explains why italy still struggle today but are probably too good to go down.

- Year - the ranking is based on games played throughout the year. England's and to a lesser extent France's general higher ranking reflects their better performance in World Cups (Except the one held in England) and against Southern Hemisphere nations in the autumn internations.
-  However, that does not always translate into success in the 6 nations.


#### Getting a column for year only and opposition without the 'V'

In [8]:
# Creates two empty lists (year and opponents) to store extracted values.
year = []
opponents = []


#  Extracting the Year from "Match Date"

for index, row in home_matches.iterrows(): # Loops through each row in home_matches
  date = row["Match Date"]
  year.append(date[-4:]) # Extracts the last 4 characters from "Match Date"
                          # Appends the extracted year (e.g., "2023") to the year list

#  Cleaning the "Opposition" Column

for index, row in home_matches.iterrows(): # Loops through home_matches again
  opp = row["Opposition"]
  opponents.append(opp[1:]) # Removes the first character from "Opposition"
                            # (assuming there's an unwanted character like a space, symbol, or extra character).
                            # Appends the cleaned name to opponents.

home_matches["Year"] = year
home_matches["0pponents"] = opponents


In [9]:
rating.head()

,Member Union,Tier,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,ENGLAND,1,High Performance,93.99,86.59,83.27,78.61,85.57,80.87,80.72,82.48,81.58,83.90,85.70,85.68,79.76,90.47,90.88,86.23,88.73
1,IRELAND,1,High Performance,80.53,82.61,80.03,84.67,78.67,78.51,84.60,81.79,80.65,80.22,79.33,82.13,81.17,84.63,86.40,91.18,84.24
2,WALES,1,High Performance,76.92,76.91,82.54,76.87,74.17,81.03,79.89,77.04,79.61,78.39,80.19,80.70,83.49,82.56,82.09,87.26,79.77
3,FRANCE,1,High Performance,82.71,83.70,86.10,85.87,80.36,79.51,82.41,81.66,84.70,85.07,80.87,80.07,79.77,80.14,78.11,77.34,85.66
4,SCOTLAND,1,High Performance,76.36,74.65,73.52,76.73,76.70,76.75,76.59,81.20,76.20,75.83,76.56,75.16,78.01,80.71,84.13,81.86,81.21


In [10]:
home_matches.head()

,Team,Diff,Opposition,Ground,Match Date,Year,0pponents
0,Wales,8,v Italy,Millennium Stadium,1 Feb 2014,2014,Italy
1,France,2,v England,Stade de France,1 Feb 2014,2014,England
2,France,-5,v Wales,Stade de France,1 Feb 2019,2019,Wales
3,Ireland,7,v Scotland,Lansdowne Road,1 Feb 2020,2020,Scotland
4,Wales,42,v Italy,Millennium Stadium,1 Feb 2020,2020,Italy


Below creates two new columns for the home standing and away standing.


In [11]:
# Create empty lists to store home and away team standings.

standing_home = []
standing_away = []


# Function to Get Standings
# Takes team name and year as inputs.
# Returns the team’s standing from the rating DataFrame.


def get_standing(team, year):
  # the numbers I assign here corelate the team to the index in the standing dataframe
  number = 0
  if team == "England":
    number = 0
  elif team == "Ireland":
    number = 1
  elif team == "Wales":
    number = 2
  elif team == "France":
    number = 3
  elif team == "Scotland":
    number = 4
  else:
    number = 5


# Passing the year I can now locate in the table rating for a team and year
# and return the value


  standing = rating.at[number, year]
  return standing


# I run this function over all the rows in the matches dataframe
for index, row in home_matches.iterrows():
  standing_home.append(get_standing(row["Team"], row["Year"]))
  standing_away.append(get_standing(row["0pponents"], row["Year"]))

# Addig the data to the home_matches dataframe

home_matches["home_standing"] = standing_home
home_matches["away_standing"] = standing_away


In [12]:
standing_home = []
standing_away = []

def get_standing(team, year):


SyntaxError: incomplete input (<ipython-input-12-7ce9aaa786d9>, line 4)

In [13]:
home_matches.head(20)

,Team,Diff,Opposition,Ground,Match Date,Year,0pponents,home_standing,away_standing
0,Wales,8,v Italy,Millennium Stadium,1 Feb 2014,2014,Italy,80.19,73.13
1,France,2,v England,Stade de France,1 Feb 2014,2014,England,80.87,73.13
2,France,-5,v Wales,Stade de France,1 Feb 2019,2019,Wales,77.34,72.76
3,Ireland,7,v Scotland,Lansdowne Road,1 Feb 2020,2020,Scotland,84.24,70.88
4,Wales,42,v Italy,Millennium Stadium,1 Feb 2020,2020,Italy,79.77,70.88
5,Ireland,10,v England,Lansdowne Road,1 Mar 2015,2015,England,82.13,72.05
6,Scotland,12,v Wales,Murrayfield,10 Feb 2007,2007,Wales,76.73,72.79
7,England,13,v Italy,Twickenham,10 Feb 2007,2007,Italy,78.61,72.79
8,Italy,-4,v England,Rome,10 Feb 2008,2008,England,73.29,73.29
9,Ireland,-6,v England,Lansdowne Road,10 Feb 2013,2013,England,80.22,76.24


In [14]:
home_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Team           255 non-null    object 
 1   Diff           255 non-null    int64  
 2   Opposition     255 non-null    object 
 3   Ground         255 non-null    object 
 4   Match Date     255 non-null    object 
 5   Year           255 non-null    object 
 6   0pponents      255 non-null    object 
 7   home_standing  255 non-null    float64
 8   away_standing  255 non-null    float64
dtypes: float64(2), int64(1), object(6)
memory usage: 18.1+ KB


In [15]:
home_matches['Match Date'] = home_matches['Match Date'].astype('datetime64[ns]')

In [16]:
home_matches.head()

,Team,Diff,Opposition,Ground,Match Date,Year,0pponents,home_standing,away_standing
0,Wales,8,v Italy,Millennium Stadium,2014-02-01,2014,Italy,80.19,73.13
1,France,2,v England,Stade de France,2014-02-01,2014,England,80.87,73.13
2,France,-5,v Wales,Stade de France,2019-02-01,2019,Wales,77.34,72.76
3,Ireland,7,v Scotland,Lansdowne Road,2020-02-01,2020,Scotland,84.24,70.88
4,Wales,42,v Italy,Millennium Stadium,2020-02-01,2020,Italy,79.77,70.88


In [17]:
home_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Team           255 non-null    object        
 1   Diff           255 non-null    int64         
 2   Opposition     255 non-null    object        
 3   Ground         255 non-null    object        
 4   Match Date     255 non-null    datetime64[ns]
 5   Year           255 non-null    object        
 6   0pponents      255 non-null    object        
 7   home_standing  255 non-null    float64       
 8   away_standing  255 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 18.1+ KB


## Adding Weather Data

- The idea behind this as to see if the weather had a big impact on the game result. A little bit of rugby knowledge is needed here but I was particularly interested to see if rain was more advantageous to teams that traditionally have relied on their forwards such as England and disadvantageous o teams that have relied on their backs like Wales. I had an API collect data for average precipitation in mm but it returned all data as zeros for each game and I know this is false. I then checked for a strung of days in Scotland again returning zeros so there must be an error in the dataset maybe? I then decided to use temp, windspeed(affects accuracy of trajectory?) And pressur as an indicator of rain.


## Previously Played

- The six nations is played:
        - Consecutively over two weekends.
        - Rest Week
        - One weekend
        - Rest Week
        - Consecutively over two weekends

  - It is tough game and injuries are common to consider the scenario.
  - England play Wales the second weekend of the consecutive cycle.
  - The previous week Wales played Ireland and England played Italy.
  - It is most likely both will have injuries but England would have likely rested a few key playes as they have never lost to Italy.
  - Wales will have played their full strength team and I would suspect will be worse off because of it.
  - I therefore would like the model to know the previous playing history of the teams. For thi I turn march date into a dtype date/time.


  - I then work thisout- this took me a long time to code and I am sure there is better way!.




In [18]:
home_matches["home_prev"] = ""
home_matches["away_prev"] = ""

game_played = False
second_cycle = True


for row in home_matches.itertuples():
  if second_cycle == True:
    if not game_played:
      home_matches.loc[row.Index, 'home_prev'] = 'free'
      home_matches.loc[row.Index, 'away_prev'] = 'free'

    else:
      if (row.Index - 1) in home_matches.index:
        home_matches.loc[row.Index - 1, 'home_prev'] = home_prev
        home_matches.loc[row.Index - 1, 'away_prev'] = away_prev

  second_cycle = True
  game_played = False

  match_day = row[5] - DT.timedelta(days=1)
  week_ago = match_day - DT.timedelta(days=9)


  home_team = row[1]
  away_team = row[7]


  home_prev = ""
  away_prev = ""

  for prev_match in home_matches.itertuples():
    if week_ago <= prev_match[5] <= match_day:
      if prev_match[1] == home_team:
        home_prev = prev_match[7]
        game_played = True

      elif prev_match[7] == home_team:
        home_prev = prev_match[1]
        game_played = True

    if prev_match[7] == away_team:
      away_prev = prev_match[1]
      game_played = True

    elif prev_match[1] == away_team:
      away_prev = prev_match[7]
      game_played = True

## Checking by getting them in data order and looking over..

In [19]:
home_matches = home_matches.groupby('Year').filter(lambda g: any(g['Year'] != "2020")).sort_values(by="Match Date")
home_matches.head(50)

,Team,Diff,Opposition,Ground,Match Date,Year,0pponents,home_standing,away_standing,home_prev,away_prev
62,France,18,v Ireland,Stade de France,2004-02-14,2004,Ireland,82.71,71.88,,Scotland
61,Wales,13,v Scotland,Millennium Stadium,2004-02-14,2004,Scotland,76.92,71.88,,Wales
74,Italy,-41,v England,Rome,2004-02-15,2004,England,71.88,71.88,,Scotland
129,France,25,v Italy,Stade de France,2004-02-21,2004,Italy,82.71,71.88,Ireland,England
128,Scotland,-22,v England,Murrayfield,2004-02-21,2004,England,76.36,71.88,,Scotland
138,Ireland,21,v Wales,Lansdowne Road,2004-02-22,2004,Wales,80.53,71.88,,Scotland
221,Italy,6,v Scotland,Rome,2004-03-06,2004,Scotland,71.88,71.88,,Wales
222,England,-6,v Ireland,Twickenham,2004-03-06,2004,Ireland,93.99,71.88,,Scotland
229,Wales,-7,v France,Millennium Stadium,2004-03-07,2004,France,76.92,71.88,,Ireland
123,Ireland,16,v Italy,Lansdowne Road,2004-03-20,2004,Italy,80.53,71.88,,England


## Function that creates train/test datasets based on input of year.

- Also removes columns that are not needed for training data.
I did this so I could train the model on all years apart from one and then test the remaining year.
- I could then  compare the absolute mean error of the model for that year against predictions I have mada in the past, to see if I ( thinking I know best with my knowledge of rugby and with a massive English bias) was better than the model.
- Worth mentioning - I have never predicted anything less than an English Grand Slam, and given that they have won 2 in the last 20 years, you can tell I am really good at this.

In [20]:
# function takes datafram and year as inputs

def create_training_test_data(year, matches):

  # Group df by input year and assign selected year to test data.
  # All other years will be used as training data.

  year = str(year)
  train_full = matches.groupby('Year').filter(lambda g: any(g['Year'] != year)).copy()
  test_full = matches.groupby('Year').filter(lambda g: any(g['Year'] == year)).copy()


# Drop columns that are no longer required

  train_full = train_full.drop(columns=['Opposition', 'Ground', 'Match Date', 'Year'], axis = 1)
  test_full = test_full.drop(columns=["Opposition", "Ground", "Match Date", "Year"], axis = 1)

  # Split into features and target
  X_train = train_full.drop(columns=["Diff"], axis=1)
  y_train = train_full["Diff"]


  X_test = test_full.drop(columns=["Diff"], axis=1)
  y_test = test_full["Diff"]

  # Create global variable dor cat and num cols for the pipeline later on

  global categorical_cols
  global numerical_cols


  # define cat and num cals for the column transformer in pepeline later

  categorical_cols = [col_name for col_name in X_train.columns if X_train[col_name].dtype == "object"]



  numerical_cols = [col_name for col_name in X_train.columns if X_train[col_name].dtype in ["int64", "float64"]]



  # return the train and test dfs
  return X_train, y_train, X_test, y_test



In [21]:
hello = create_training_test_data('2004', home_matches)
hello[2].head()

,Team,0pponents,home_standing,away_standing,home_prev,away_prev
62,France,Ireland,82.71,71.88,,Scotland
61,Wales,Scotland,76.92,71.88,,Wales
74,Italy,England,71.88,71.88,,Scotland
129,France,Italy,82.71,71.88,Ireland,England
128,Scotland,England,76.36,71.88,,Scotland


## Make sure the target is same format

In [22]:
target.head()

,Team,opponnents\n,home_standing\n,away_standing\n,home_prev\n,away_prev\n
0,Italy,France,70.88,85.30,free,free
1,England,Scotland,89.49,80.82,free,free
2,Wales,Ireland,79.36,84.65,free,free
3,England,Italy,89.49,70.88,Scotland,France
4,Scotland,Wales,80.82,79.36,England,Ireland


In [23]:
target.columns = ["Team", "opponents", "home_standing", "away_standing", "home_prev", "away_prev"]

In [24]:
target.head()

,Team,opponents,home_standing,away_standing,home_prev,away_prev
0,Italy,France,70.88,85.30,free,free
1,England,Scotland,89.49,80.82,free,free
2,Wales,Ireland,79.36,84.65,free,free
3,England,Italy,89.49,70.88,Scotland,France
4,Scotland,Wales,80.82,79.36,England,Ireland


## Create Pipeline

- This pipeline takes the numerical data and performs a simple scaler to it.
- There is no need to impute values as I know there are no missing values.
- One hot encoder is used to deal with the categorical data to avoid any clustering from using label encoding.

In [25]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [26]:
# Function will take the model as input and return the pipleine

def create_pipeline(model):
  numerical_transformer = StandardScaler()
  categorical_transformer = OneHotEncoder(handle_unknown='ignore')

  preprocessor = ColumnTransformer(
      transformers=[
          ('num', numerical_transformer, numerical_cols),
          ('cat', categorical_transformer, categorical_cols)
      ])


  pipe_line = Pipeline(steps=[('preprocessor', preprocessor),
                             ('model', model)])

  return pipe_line

## First model will use simple regression

- All models follow the same pattern for creating the model and then a train/test dataset for each year.
- The Absolute Mean Error was then calculated for each dataset and then averaged out.

In [27]:
# Create a list with all years

x = range(2004, 2020)

In [28]:
# create regression model
model = LinearRegression()


# Create empty dictionary to store MAE error and year
year_results_regression = {}


# for year in years

for n in x:
  # Create data set for each year
  X_train, y_train, X_test, y_test = create_training_test_data(n, home_matches)

  # Create pipeline with model
  clf = create_pipeline(model)

  # fit the data to the model
  clf.fit(X_train, y_train)

  # predict using the model
  preds = clf.predict(X_test)

  # Add to the dictionary the results for that year

  mae = mean_absolute_error(y_test, preds)
  year_results_regression[n] = mae


# Print out dictionaru showing MAE for each year
print(year_results_regression)

# Print out average of all erros
print(sum(year_results_regression.values()) / float(len(year_results_regression)))


{2004: 12.759244329121968, 2005: 10.357036281097047, 2006: 13.55732231606167, 2007: 14.88728245108185, 2008: 11.422528858148388, 2009: 7.788122583268633, 2010: 8.279677836873482, 2011: 9.71298863578299, 2012: 6.472932136953811, 2013: 12.870721274048327, 2014: 11.438661193492063, 2015: 9.12235199675386, 2016: 11.108916486582944, 2017: 9.771458251229195, 2018: 7.229780870086347, 2019: 9.656047439029717}
10.40219205872577


## Lasso Regreesion

In [29]:
from sklearn import linear_model

# create regression lasso model
model = linear_model.Lasso(alpha=1.5)


# create empty dictionary to store MAE error and year
year_results_lasso = {}


# for year in years

for n in x:
  # Create data set for each year
  X_train, y_train, X_test, y_test = create_training_test_data(n, home_matches)

  # Create pipeline with model
  clf = create_pipeline(model)

  # fit the data to the model
  clf.fit(X_train, y_train)

  # Predit using the model
  preds = clf.predict(X_test)

  # Add to the dictionary the results for that year
  mae = mean_absolute_error(y_test, preds)
  year_results_lasso[n] = mae



print(year_results_lasso)

print(sum(year_results_lasso.values()) / float(len(year_results_lasso)))


{2004: 16.533825675719303, 2005: 13.066719289757776, 2006: 9.68609547467483, 2007: 13.515999030246833, 2008: 12.814315380301856, 2009: 10.322288499988982, 2010: 8.518928116269601, 2011: 9.04965645452965, 2012: 8.898313790867856, 2013: 10.447841106082873, 2014: 14.080771117971853, 2015: 10.397107299773701, 2016: 10.881903528040336, 2017: 11.903900320118137, 2018: 10.148539205406092, 2019: 10.911505117567092}
11.323606837957298


# Random Forest Regression

In [31]:
from sklearn.ensemble import RandomForestRegressor

# create regression lasso model
model = RandomForestRegressor(max_depth=3)

# create empty dictionary to store MAE error and year
year_results_forest_regressor = {}

# for year in years

for n in x:
  # Create data set for each year
  X_train, y_train, X_test, y_test = create_training_test_data(n, home_matches)

  # Create pipeline with model
  clf = create_pipeline(model)

  # fit the data to the model
  clf.fit(X_train, y_train)

  # predict using model
  preds = clf.predict(X_test)


  # Add to the dictionary the results foe that year
  mae = mean_absolute_error(y_test, preds)
  year_results_forest_regressor[n] = mae


print(year_results_forest_regressor)
print(sum(year_results_forest_regressor.values()) / float(len(year_results_forest_regressor)))

{2004: 11.861471782770328, 2005: 9.891883172823823, 2006: 13.56845069964479, 2007: 12.485907589395513, 2008: 14.623035713874234, 2009: 8.982782272912939, 2010: 9.378242584375744, 2011: 9.139729719933825, 2012: 5.9927196382502474, 2013: 12.502853480450339, 2014: 12.643175077116659, 2015: 7.7102388327833875, 2016: 10.739947150662704, 2017: 9.054440000031246, 2018: 6.468345382416136, 2019: 11.756376889896762}
10.424974999208667


In [47]:
predicted = clf.predict(target)
target['Predicted Result'] = predicted
target['Actual'] = ['-40', '-5', '5', '23', '-1', '-2', '-38', '16', 'TBD', 'TBD', 'TBD', 'TBD', 'TBD', 'TBD', 'TBD']
target.head(15)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [48]:
print(target.dtypes)  # Check column names


Team              object
opponents         object
home_standing    float64
away_standing    float64
home_prev         object
away_prev         object
Opponents          int64
0pponents          int64
dtype: object


In [49]:
target.head()

,Team,opponents,home_standing,away_standing,home_prev,away_prev,Opponents,0pponents
0,Italy,France,70.88,85.30,free,free,0,0
1,England,Scotland,89.49,80.82,free,free,0,0
2,Wales,Ireland,79.36,84.65,free,free,0,0
3,England,Italy,89.49,70.88,Scotland,France,0,0
4,Scotland,Wales,80.82,79.36,England,Ireland,0,0


In [36]:
print(clf.feature_names_in_)


['Team' '0pponents' 'home_standing' 'away_standing' 'home_prev'
 'away_prev']


In [39]:
missing_cols = set(clf.feature_names_in_) - set(target.columns)
print("Missing columns:", missing_cols)


Missing columns: {'0pponents'}


In [40]:
for col in missing_cols:
    target[col] = 0  # Fill missing columns with default values (adjust as needed)


In [41]:
target_transformed = preprocessor.transform(target)  # If using a transformer


NameError: name 'preprocessor' is not defined

In [42]:
# Check and fix column name typo
if '0pponents' in target.columns:
    target.rename(columns={'0pponents': 'Opponents'}, inplace=True)

# Ensure target has all required columns
missing_cols = set(clf.feature_names_in_) - set(target.columns)
for col in missing_cols:
    target[col] = 0  # Add missing columns with default values

# Ensure correct data preprocessing
target_transformed = preprocessor.transform(target)  # If using a transformer

# Predict
predicted = clf.predict(target_transformed)

# Store predictions
target['Predicted Result'] = predicted

# Add actual values
target['Actual'] = ['-40', '-5', '5', '23', '-1', '-2', '-38', '16', 'TBD', 'TBD', 'TBD', 'TBD', 'TBD', 'TBD', 'TBD']

# Display
target.head(15)


NameError: name 'preprocessor' is not defined

In [43]:
print(clf)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['home_standing',
                                                   'away_standing']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Team', '0pponents',
                                                   'home_prev',
                                                   'away_prev'])])),
                ('model', RandomForestRegressor(max_depth=3))])


In [44]:
target_transformed = clf.named_steps['preprocessor'].transform(target)


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [45]:
if hasattr(clf, "named_steps") and "preprocessor" in clf.named_steps:
    preprocessor = clf.named_steps["preprocessor"]  # Extract preprocessor from pipeline
    target_transformed = preprocessor.transform(target)
else:
    target_transformed = target  # If no preprocessor, use original data

predicted = clf.predict(target_transformed)
target["Predicted Result"] = predicted


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

## XGBoost with GridSearch

In [51]:
X_results = X_test.copy()

In [52]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error


import xgboost as xgb

model = xgb.XGBRegressor(
    learning_rate=0.05,
    colsample_bytree=0.3,
    min_child_weight=7,
    gamma=20)

# Create empty dictionary to store MAE error and year

year_results_xgBoost = {}

# for year in years

for n in x:
  # Create dataset for each year
  X_train, y_train, X_test, y_test = create_training_test_data(n, home_matches)

  # Create pipeline with model
  clf = create_pipeline(model)

  # fit the data to the model
  clf.fit(X_train, y_train)

  # Predit using the model
  preds = clf.predict(X_test)


  # Add to the dictionary the results for that year

  mae = mean_absolute_error(y_test, preds)
  year_results_xgBoost[n]= mae


  print(year_results_xgBoost)
  print(sum(year_results_xgBoost.values()) / float(len(year_results_xgBoost)))

AttributeError: 'super' object has no attribute '__sklearn_tags__'

In [53]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import numpy as np

# Ensure you have the latest version of XGBoost
print("XGBoost Version:", xgb.__version__)

# Define model with sklearn compatibility
model = xgb.XGBRegressor(
    learning_rate=0.05,
    colsample_bytree=0.3,
    min_child_weight=7,
    gamma=20,
    enable_categorical=True  # Helps with categorical features
)

# Create empty dictionary to store MAE error and year
year_results_xgBoost = {}

# Loop through years (assuming `x` is a list of years)
for n in x:
    # Create dataset for each year
    X_train, y_train, X_test, y_test = create_training_test_data(n, home_matches)

    # Convert to NumPy array (fixes some compatibility issues)
    X_train = np.array(X_train)
    X_test = np.array(X_test)

    # Create pipeline with model (ensure `create_pipeline()` is correctly implemented)
    clf = create_pipeline(model)

    # Fit the model
    clf.fit(X_train, y_train)

    # Predict using the model
    preds = clf.predict(X_test)

    # Calculate and store MAE
    mae = mean_absolute_error(y_test, preds)
    year_results_xgBoost[n] = mae

    print(year_results_xgBoost)
    print(sum(year_results_xgBoost.values()) / float(len(year_results_xgBoost)))



XGBoost Version: 2.1.3


ValueError: Specifying the columns using strings is only supported for dataframes.

In [56]:
from sklearn.pipeline import make_pipeline

In [57]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import pandas as pd  # Keep data in DataFrame format

# Ensure you have the latest version of XGBoost
print("XGBoost Version:", xgb.__version__)

# Define model
model = make_pipeline(xgb.XGBRegressor(
    learning_rate=0.05,
    colsample_bytree=0.3,
    min_child_weight=7,
    gamma=20,
    enable_categorical=True  # Helps if categorical features exist
))

# Dictionary to store results
year_results_xgBoost = {}

# Loop through years (assuming `x` is a list of years)
for n in x:
    # Create dataset for each year
    X_train, y_train, X_test, y_test = create_training_test_data(n, home_matches)

    # ✅ Keep X_train and X_test as Pandas DataFrames (NO np.array conversion!)
    if not isinstance(X_train, pd.DataFrame):
        X_train = pd.DataFrame(X_train)

    if not isinstance(X_test, pd.DataFrame):
        X_test = pd.DataFrame(X_test)

    # Create pipeline
    clf = create_pipeline(model)

    # Fit the model
    clf.fit(X_train, y_train)

    # Predict using the model
    preds = clf.predict(X_test)

    # Calculate and store MAE
    mae = mean_absolute_error(y_test, preds)
    year_results_xgBoost[n] = mae

    print(year_results_xgBoost)
    print(sum(year_results_xgBoost.values()) / float(len(year_results_xgBoost)))


XGBoost Version: 2.1.3


AttributeError: 'super' object has no attribute '__sklearn_tags__'